In [1]:
%pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import annotations
import itertools
from typing import List, Optional

In [3]:
class TreeNode:
    _id_iter = itertools.count()
    def __init__(
        self,
        *,
        player: Optional[int] = None,      # 1 or 2; None for the root
        action: Optional[int] = None,      # apples taken at this node
        remaining: int = 0,                # apples left after this node’s action
    ):
        self.id: int = next(self._id_iter)
        self.player: Optional[int] = player          # None for the root
        self.action: Optional[int] = action
        self.remaining: int = remaining

        # These are only set for terminal nodes (after both players have moved)
        self.payoff: Optional[tuple[int, int]] = None

        # For pretty printing / graph traversal
        self.children: List["TreeNode"] = []

    def __repr__(self) -> str:
        if self.player is None:
            return f"Root(remaining={self.remaining})"
        else:
            return (
                f"P{self.player} takes {self.action} "
                f"(remaining={self.remaining})"
            )


In [4]:
def build_sharing_game_tree(total_apples: int = 2) -> TreeNode:
    """
    Returns the root of a tree that represents the extensive form of
    the sharing game with `total_apples` apples and two players.
    The first player moves, then the second player.  Each may take
    any number of apples from the remaining pile (including zero).
    """
    root = TreeNode(remaining=total_apples)

    def _add_children(node: TreeNode, next_player: int) -> None:
        """Recursively add children for `next_player`."""
        if node.remaining == 0:
            # No apples left – terminal node
            node.payoff = (node.take_by_player1, node.take_by_player2)
            return

        if next_player == 1:
            # Player 1 chooses from 0..remaining
            for a in range(node.remaining + 1):
                child = TreeNode(
                    player=1,
                    action=a,
                    remaining=node.remaining - a,
                )
                # Carry forward the opponent’s earlier choice (if any)
                child.take_by_player1 = a
                child.take_by_player2 = getattr(node, "take_by_player2", 0)

                node.children.append(child)
                _add_children(child, next_player=2)          # Now player 2 moves

        else:   # next_player == 2
            for b in range(node.remaining + 1):
                child = TreeNode(
                    player=2,
                    action=b,
                    remaining=node.remaining - b,
                )
                child.take_by_player2 = b
                child.take_by_player1 = getattr(node, "take_by_player1", 0)

                node.children.append(child)
                # After player 2 there are no more moves – terminal state
                child.payoff = (child.take_by_player1, child.take_by_player2)

    _add_children(root, next_player=1)
    return root


In [5]:
def print_tree(node: TreeNode, indent: str = "") -> None:
    if node.player is None:
        label = f"Root (remaining={node.remaining})"
    else:
        label = f"P{node.player} takes {node.action} " \
                f"(remaining={node.remaining})"

    if node.payoff is not None:
        label += f"  ➜ Payoff: {node.payoff}"

    print(indent + label)

    for child in node.children:
        print_tree(child, indent=indent + "  ")

In [6]:
def normal_form_markdown(root: TreeNode,
                         max_apples: int = 2) -> str:
    """
    Convert an extensive-form tree into a Markdown table that shows the
    normal-form payoff matrix.

    Parameters
    ----------
    root : TreeNode
        The root of the game tree (the one produced by `build_sharing_game_tree`).
    max_apples : int, optional
        Highest action value that any player can take.  In the standard game it
        is equal to the total number of apples.

    Returns
    -------
    str
        A Markdown string that you can paste into a Jupyter cell and display.
    """
    # ------------------------------------------------------------------
    # 1) Collect every terminal payoff
    # ------------------------------------------------------------------
    payoff_map = {}          # key: (a1,a2)  value: (p1,p2)

    def dfs(node: TreeNode):
        if node.payoff is not None:
            a1 = getattr(node, "take_by_player1", 0)
            a2 = getattr(node, "take_by_player2", 0)
            payoff_map[(a1, a2)] = node.payoff
            return
        for child in node.children:
            dfs(child)

    dfs(root)

    # ------------------------------------------------------------------
    # 2) Build the Markdown table
    # ------------------------------------------------------------------
    actions = list(range(max_apples + 1))

    # header   = ["|"] + [f"P2:{a}" for a in actions] + ["|"]
    header   = ""
    separator= "|" + "|".join(["---"] * len(actions)) + "|"
    rows     = [header, separator]

    for a1 in actions:
        row = [f"|P1:{a1}"]
        for a2 in actions:
            payoff = payoff_map.get((a1, a2))
            cell   = f"({payoff[0]},{payoff[1]})" if payoff else "-"
            row.append(cell)
        row.append("|")
        rows.append("".join(row))

    return "\n".join(rows)

In [7]:
root = build_sharing_game_tree(total_apples=2)
print("=== ASCII representation of the extensive‑form tree ===\n")
print_tree(root)

md_table = normal_form_markdown(root, max_apples=2)
print(md_table)


=== ASCII representation of the extensive‑form tree ===

Root (remaining=2)
  P1 takes 0 (remaining=2)
    P2 takes 0 (remaining=2)  ➜ Payoff: (0, 0)
    P2 takes 1 (remaining=1)  ➜ Payoff: (0, 1)
    P2 takes 2 (remaining=0)  ➜ Payoff: (0, 2)
  P1 takes 1 (remaining=1)
    P2 takes 0 (remaining=1)  ➜ Payoff: (1, 0)
    P2 takes 1 (remaining=0)  ➜ Payoff: (1, 1)
  P1 takes 2 (remaining=0)  ➜ Payoff: (2, 0)

|---|---|---|
|P1:0(0,0)(0,1)(0,2)|
|P1:1(1,0)(1,1)-|
|P1:2(2,0)--|
